In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder  

In [2]:
df = pd.read_csv("santander_limpio_filtrado_y_data_engeniering.csv")

In [3]:
df.shape

(319400, 22)

In [4]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

df['sexo_n'] = encoder.fit_transform(df['sexo'])

df['canal_entrada_n'] = encoder.fit_transform(df['canal_entrada'])

df['segmento_n'] = encoder.fit_transform(df['segmento'])

In [5]:
df.columns

Index(['sexo', 'age', 'antiguedad', 'canal_entrada', 'renta', 'segmento',
       'cuenta_ahorros', 'cuenta_corriente', 'credito_rapido', 'cuenta_nomina',
       'cuenta_joven', 'cuenta_adulto_mayor', 'cuenta_apertura',
       'cuenta_pagos', 'cuenta_debitos', 'deposito_largo_plazo', 'ecuenta',
       'hipotecas', 'ahorro_programado', 'prestamo_libre_inversion',
       'tarjeta_credito', 'credito_vivienda', 'sexo_n', 'canal_entrada_n',
       'segmento_n'],
      dtype='object')

In [6]:
var_categoricas_2 = []
var_numericas_2 = []
var_fechas_2 = []

for col in df.columns:
    if pd.api.types.is_datetime64_any_dtype(df[col]):
        var_fechas_2.append(col)
    elif pd.api.types.is_numeric_dtype(df[col]):
        var_numericas_2.append(col)
    elif df[col].dtype == 'object':
        var_categoricas_2.append(col)

print("Variables de tipo fecha:")
print(var_fechas_2)

print("Variables categóricas:")
print(var_categoricas_2)

print("Variables numéricas:")
print(var_numericas_2)

Variables de tipo fecha:
[]
Variables categóricas:
['sexo', 'canal_entrada', 'segmento']
Variables numéricas:
['age', 'antiguedad', 'renta', 'cuenta_ahorros', 'cuenta_corriente', 'credito_rapido', 'cuenta_nomina', 'cuenta_joven', 'cuenta_adulto_mayor', 'cuenta_apertura', 'cuenta_pagos', 'cuenta_debitos', 'deposito_largo_plazo', 'ecuenta', 'hipotecas', 'ahorro_programado', 'prestamo_libre_inversion', 'tarjeta_credito', 'credito_vivienda', 'sexo_n', 'canal_entrada_n', 'segmento_n']


In [7]:
df.drop(['sexo', 'canal_entrada', 'segmento'], axis=1, inplace=True)

In [8]:
targets = ['tarjeta_credito', 'ecuenta', 'deposito_largo_plazo']
print(set(var_numericas_2) & set(targets))

var_numericas_2 = [col for col in var_numericas_2 if col not in targets] # asegura que no estén las target 
print(set(var_numericas_2) & set(targets))


{'tarjeta_credito', 'deposito_largo_plazo', 'ecuenta'}
set()


In [9]:
scaler = StandardScaler()
df[var_numericas_2] = scaler.fit_transform(df[var_numericas_2])

In [10]:
X = df.drop(columns=targets)
y = df[targets].astype(int)  # aseguramos enteros

In [11]:
list(X.columns)

['age',
 'antiguedad',
 'renta',
 'cuenta_ahorros',
 'cuenta_corriente',
 'credito_rapido',
 'cuenta_nomina',
 'cuenta_joven',
 'cuenta_adulto_mayor',
 'cuenta_apertura',
 'cuenta_pagos',
 'cuenta_debitos',
 'hipotecas',
 'ahorro_programado',
 'prestamo_libre_inversion',
 'credito_vivienda',
 'sexo_n',
 'canal_entrada_n',
 'segmento_n']

In [12]:
y

,tarjeta_credito,ecuenta,deposito_largo_plazo
0,0,1,0
1,0,1,0
2,1,0,0
3,0,1,0
4,0,1,0
...,...,...,...
319395,0,1,0
319396,0,1,0
319397,1,0,0
319398,1,1,0


In [13]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier  
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


modelo_base = XGBClassifier(n_estimators=100, learning_rate=0.1, eval_metric='logloss')

# 4. Entrenamiento con Multi-output wrapper
# ---------------------------
modelo_multi = MultiOutputClassifier(estimator=modelo_base)
modelo_multi.fit(X_train, y_train)


# # ---------------------------
# # 6. Predicción y evaluación
# # ---------------------------
y_pred = modelo_multi.predict(X_test)

# # Reporte para cada variable objetivo
for i, target in enumerate(targets):
    print(f"\n📌 Resultados para: {target}")
    print(classification_report(y_test[target], y_pred[:, i], zero_division=1))





📌 Resultados para: tarjeta_credito
              precision    recall  f1-score   support

           0       0.78      0.86      0.82     43212
           1       0.63      0.49      0.55     20668

    accuracy                           0.74     63880
   macro avg       0.70      0.67      0.68     63880
weighted avg       0.73      0.74      0.73     63880


📌 Resultados para: ecuenta
              precision    recall  f1-score   support

           0       0.61      0.48      0.54     22569
           1       0.75      0.84      0.79     41311

    accuracy                           0.71     63880
   macro avg       0.68      0.66      0.66     63880
weighted avg       0.70      0.71      0.70     63880


📌 Resultados para: deposito_largo_plazo
              precision    recall  f1-score   support

           0       0.80      0.92      0.85     44598
           1       0.71      0.45      0.55     19282

    accuracy                           0.78     63880
   macro avg       0.75

In [14]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(255520, 19)
(63880, 19)
(255520, 3)
(63880, 3)


In [ ]:
import pickle

with open('modelo_xgb_v1.pkl', 'wb') as f:
    pickle.dump({
        'modelo': modelo_multi,
        'targets': targets,
        'features': X.columns.tolist()
    }, f)

## RandomizedSearchCV

In [15]:
from scipy.stats import randint
import numpy as np
from sklearn.model_selection import RandomizedSearchCV


xg_rsv = {"estimator__n_estimators": np.arange(100, 600, 100),
          "estimator__max_depth": [3, 5, 10, 15, 20, 30],
          "estimator__learning_rate": [0.01, 0.1, 0.5, 1], 
          "estimator__gamma": [0, 0.1, 0.4],
          "estimator__subsample": [0.6, 0.8],
          "estimator__colsample_bytree": [0.6, 0.8, 1.0],
          "estimator__scale_pos_weight": [1, 1.5, 1.87, 2, 3]
}
# Crear el modelo base
# rf_model = RandomForestClassifier(random_state=42)
# multi_rf = MultiOutputClassifier(modelo_base)

# multi_xgb = MultiOutputClassifier(modelo_base)

# Optimización con RandomizedSearchCV
# XGBClassifier
xg_model = RandomizedSearchCV(
    estimator=modelo_multi,
    param_distributions=xg_rsv,
    scoring="f1_micro",
    n_iter=5,        # Reducido
    cv=3,            # Reducido
    n_jobs=3,        # Importante: solo 1 job
    verbose=True,
    random_state=42
)

# Ajustar el modelo
xg_model.fit(X_train, y_train)


print("Mejores hiperparámetros encontrados:")
print(xg_model.best_params_)
print("Mejor f1_score promedio:", xg_model.best_score_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/home/sytic/miniconda3/envs/data-science/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Mejores hiperparámetros encontrados:
{'estimator__subsample': 0.6, 'estimator__scale_pos_weight': 1, 'estimator__n_estimators': 500, 'estimator__max_depth': 30, 'estimator__learning_rate': 0.1, 'estimator__gamma': 0.1, 'estimator__colsample_bytree': 0.8}
Mejor f1_score promedio: 0.8483998579137636


In [16]:
best_model = xg_model.best_estimator_

# Hacer predicciones en el conjunto de prueba
y_best_pred = best_model.predict(X_test)

# Evaluar

for i, target in enumerate(targets):
    print(f"\n📌 Resultados para: {target}")
    print(classification_report(y_test[target], y_best_pred[:, i], zero_division=1))


📌 Resultados para: tarjeta_credito
              precision    recall  f1-score   support

           0       0.91      0.94      0.93     43212
           1       0.87      0.82      0.84     20668

    accuracy                           0.90     63880
   macro avg       0.89      0.88      0.89     63880
weighted avg       0.90      0.90      0.90     63880


📌 Resultados para: ecuenta
              precision    recall  f1-score   support

           0       0.91      0.86      0.89     22569
           1       0.93      0.95      0.94     41311

    accuracy                           0.92     63880
   macro avg       0.92      0.91      0.91     63880
weighted avg       0.92      0.92      0.92     63880


📌 Resultados para: deposito_largo_plazo
              precision    recall  f1-score   support

           0       0.94      0.97      0.96     44598
           1       0.92      0.87      0.89     19282

    accuracy                           0.94     63880
   macro avg       0.93

In [23]:
import joblib
modelo_completo = {
    'modelo': xg_model.best_estimator_,
    'features': X.columns.tolist(),
    'targets': targets,
}

# Guardar con nombre descriptivo
nombre_archivo = f'modelo_xgb_optimizado_final.pkl'

with open(nombre_archivo, 'wb') as f:
    joblib.dump(modelo_completo, f, compress=9)